
 # FINAL YEAR PROJECT #
 
# SUPERVISOR GLADYS MANGE  #

 # 21/04428 NJOROGE HALDLINE MUKAMI  #
 
#  BDS 4105  #

 # Maize Crop Yield Prediction and Advisory System for Kenya .





This project aims to develop a maize crop yield prediction and advisory system for Kenya using historical agricultural and climate-related data. The system predicts expected maize yield and provides simple, data-driven advisory insights to support farmers and agricultural stakeholders.


''  IMPORT LIBRARIES (CODE) ''

In [13]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib


## Introduction

This notebook implements a maize crop yield prediction system for Kenya.
The objective is to clean agricultural production data, train machine
learning models, and generate insights that can support farmers and
policy makers.

The system is designed to be simple, scalable, and suitable for
integration with satellite data and a web-based advisory platform.


''  CELL 2  LOAD DATA (CODE)''

In [17]:
import pandas as pd

# Path to FAOSTAT CSV file
faostat_path = r"C:\Users\ICTADMIN\Desktop\FINAL YEAR PROJECT\Maize-Yield -System\FAOSTAT_data_en_12-30-2025 (2).csv"

# Path to Maize Excel file
maize_xlsx_path = r"C:\Users\ICTADMIN\Desktop\FINAL YEAR PROJECT\Maize-Yield -System\Maize-Production-2012-2020-Combined (1).xlsx"

# Load datasets
faostat_df = pd.read_csv(faostat_path)
maize_df = pd.read_excel(maize_xlsx_path)

# Preview maize dataset
maize_df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,NaN,2012,NaN,NaN,2013,NaN,NaN,2014,NaN,NaN,...,NaN,2017,NaN,NaN,2018,NaN,NaN,2020,NaN,NaN
1,COUNTY,Harvested Area (HA),Production (MT),Yield (MT/HA),Harvested Area (HA),Production (MT),Yield (MT/HA),Harvested Area (HA),Production (MT),Yield (MT/HA),...,Yield (MT/HA),Area (HA) 2017,Production (MT) 2017,Yield(MT/HA),Area Ha 2018,Quantity Ton 2018,Yield(MT/HA),Area (Ha) 2020,Production (MT) 2020,Yield (MT/HA)
2,Baringo,39753,71866.62,1.807829,29117,55805.04,1.916579,34960,34959,0.999971,...,2.186021,33308,45764,1.373964,37658,58475.6,1.552807,47437,68374.39,1.441373
3,Bomet,32697,73277.82,2.241118,30620,72235.8,2.359105,30940,53162,1.718229,...,1.410287,33792,56601,1.674982,33291,58336.81,1.75233,38163,73609.65,1.928822
4,Bungoma,96209,262381.05,2.727199,92705,221586.03,2.390227,98074,267302,2.725513,...,2.989396,95323,289290,3.034839,93484,295481.1,3.160767,87960,317912.33,3.614283


## Data Sources

Two datasets are used in this project:

1. FAOSTAT dataset – used for validation and reporting.
2. Maize production dataset (2012–2020) – used for machine learning
   model training.

The maize production dataset contains yield, area harvested, and
production statistics across multiple years.


'' CELL 4 — CLEAN EXCEL STRUCTURE (CODE)''

In [21]:
# Remove completely empty columns
maize_df = maize_df.dropna(axis=1, how='all')

# Rename first column to County
maize_df = maize_df.rename(columns={maize_df.columns[0]: "County"})

maize_df.head()


,County,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,NaN,2012,NaN,NaN,2013,NaN,NaN,2014,NaN,NaN,...,NaN,2017,NaN,NaN,2018,NaN,NaN,2020,NaN,NaN
1,COUNTY,Harvested Area (HA),Production (MT),Yield (MT/HA),Harvested Area (HA),Production (MT),Yield (MT/HA),Harvested Area (HA),Production (MT),Yield (MT/HA),...,Yield (MT/HA),Area (HA) 2017,Production (MT) 2017,Yield(MT/HA),Area Ha 2018,Quantity Ton 2018,Yield(MT/HA),Area (Ha) 2020,Production (MT) 2020,Yield (MT/HA)
2,Baringo,39753,71866.62,1.807829,29117,55805.04,1.916579,34960,34959,0.999971,...,2.186021,33308,45764,1.373964,37658,58475.6,1.552807,47437,68374.39,1.441373
3,Bomet,32697,73277.82,2.241118,30620,72235.8,2.359105,30940,53162,1.718229,...,1.410287,33792,56601,1.674982,33291,58336.81,1.75233,38163,73609.65,1.928822
4,Bungoma,96209,262381.05,2.727199,92705,221586.03,2.390227,98074,267302,2.725513,...,2.989396,95323,289290,3.034839,93484,295481.1,3.160767,87960,317912.33,3.614283


''  RESHAPE DATA (CRITICAL STEP) (CODE) ''